In [1]:
import pandas as pd
crySS = pd.read_csv('../../nwac/425/mt-baker-ski-area-wind_speed_maximum.csv', comment='#')
crySS = crySS.drop(index = 4343)
crySS = crySS.rename(columns={'Date/Time (PST)': 'DateTime'})
crySS['DateTime'] = pd.to_datetime(crySS['DateTime'])
crySS['DateTime'] = crySS['DateTime'].astype(str)
crySS.set_index('DateTime')
crySS = crySS.iloc[::-1]
crySS = crySS.set_index('DateTime')
crySS.tail()
crySS.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4343 entries, 2022-11-01 00:00:00 to 2023-04-30 22:00:00
Data columns (total 1 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   mph (5020') Mt. Baker - Pan Dome  4325 non-null   float64
dtypes: float64(1)
memory usage: 67.9+ KB


In [3]:
from collections import deque

hourly_wind_readings = deque()
wind_dfs = []


new_df = pd.DataFrame()
new_df['Date'] = None
new_df['Max Windspeed 24hr'] = None
new_df = new_df.set_index('Date')

for idx, row in crySS.iterrows():
    date = idx.split(' ')[0]
    hour = idx.split(' ')[1]
    if hour == '00:00:00' and len(hourly_wind_readings) >= 24:
        m = -1
        for w in hourly_wind_readings:
            if w > m:
                m = w
        new_df.loc[date, 'Max Windspeed 24hr'] = m
    if len(hourly_wind_readings) >= 24:
        hourly_wind_readings.pop()
    hourly_wind_readings.appendleft(row['mph (5020\') Mt. Baker - Pan Dome'])

wind_dfs.append(new_df)
new_df.tail()

,Max Windspeed 24hr
Date,
2023-04-26,15.53
2023-04-27,21.89
2023-04-28,16.24
2023-04-29,16.94
2023-04-30,35.3


[           Max Windspeed 24hr
 Date                         
 2022-11-02              16.24
 2022-11-03               11.3
 2022-11-04               47.3
 2022-11-05              72.01
 2022-11-06              50.83
 ...                       ...
 2023-04-26              15.53
 2023-04-27              21.89
 2023-04-28              16.24
 2023-04-29              16.94
 2023-04-30               35.3
 
 [180 rows x 1 columns]]

In [9]:
final_wind = new_df
final_wind['Was High Winds 24hr'] = final_wind.apply(lambda row : 1 if row['Max Windspeed 24hr'] >= 25 else 0, axis=1)
final_wind.to_csv('../../nwac/425/wsn.csv', sep=',')
final_wind.tail()

,Max Windspeed 24hr,Was High Winds 24hr
Date,,
2023-04-26,15.53,0
2023-04-27,21.89,0
2023-04-28,16.24,0
2023-04-29,16.94,0
2023-04-30,35.3,1
